In [4]:
import os
import requests
import json
import time
from urllib.parse import quote

def get_biogrid_interactions(gene_name, access_key, search_type="searchNames", tax_id="9606",
                            include_interactors=True, max_results=10000, format="json"):
    """
    Fetches known interactions for a gene from BioGRID.

    Args:
        gene_name: The gene name to search for (e.g., "MDM2").
        access_key: Your BioGRID API access key.
        search_type: How to search for the gene ("searchNames", "searchIds", "searchSynonyms", "additionalIdentifierTypes").
        tax_id: The NCBI taxonomy ID for the organism (default: "9606" for human).
        include_interactors: Whether to include first-order interactions (default: True).
        max_results: The maximum number of results to fetch (default: 10000).
        format: The desired response format ("json" or "jsonExtended").

    Returns:
        A dictionary containing the parsed JSON response from BioGRID, or None if an error occurred.
    """
    base_url = "https://webservice.thebiogrid.org/interactions/"
    params = {
        "accesskey": access_key,
        "geneList": gene_name,
        "taxId": tax_id,
        "includeInteractors": str(include_interactors).lower(),
        "format": format,
        "max": max_results,
    }

    if search_type == "searchNames":
        params["searchNames"] = "true"
    elif search_type == "searchIds":
        params["searchIds"] = "true"
    elif search_type == "searchSynonyms":
        params["searchSynonyms"] = "true"
    elif search_type == "additionalIdentifierTypes":
        params["additionalIdentifierTypes"] = "UNIPROT"
    else:
        print("Invalid search_type. Options: searchNames, searchIds, searchSynonyms, additionalIdentifierTypes")
        return None

    try:
        url = base_url + "?" + "&".join([f"{k}={v}" for k, v in params.items()])
        response = requests.get(url)
        response.raise_for_status() # Raise HTTPError for bad responses (4xx or 5xx)
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data from BioGRID: {e}")
        return None


def extract_biogrid_interactions(biogrid_data):
    """
    Extracts relevant interaction information from the BioGRID JSON response.
     Args:
        biogrid_data: The JSON response from the BioGRID API.

    Returns:
        A list of dictionaries, each containing interaction info or an empty list if an error occurred.
    """
    interactions = []

    if not biogrid_data or "entries" not in biogrid_data:
       return interactions

    for entry in biogrid_data["entries"]:
        for interaction in entry.get("interactions", []):
            int_info = {}
            int_info['interaction_id'] = interaction.get("interactionId")
            int_info['interaction_type'] = interaction.get("interactionType")
            int_info["evidence_code"] = interaction.get("evidenceCode")

            interactors = []
            for interactor in interaction.get("interactors", []):
                interactor_info = {}
                interactor_info["entrez_gene_id"] = interactor.get("entrezGeneId")
                interactor_info["official_symbol"] = interactor.get("officialSymbol")
                interactor_info["synonyms"] = interactor.get("synonyms")
                interactor_info["organism_name"] = interactor.get("organismName")
                interactor_info["organism_tax_id"] = interactor.get("organismTaxId")
                interactors.append(interactor_info)

            int_info["interactors"] = interactors
            interactions.append(int_info)

    return interactions

from dotenv import load_dotenv

load_dotenv()

# Example Usage
gene_name = "CLOCK"
access_key = os.getenv("BIOGRID_API_KEY")
tax_id = "9606"  # Human
format = "json"
search_type = "searchNames" # searchNames or additionalIdentifierTypes
biogrid_data = get_biogrid_interactions(
    gene_name, access_key, tax_id=tax_id, format=format, search_type=search_type
)

if biogrid_data:
    extracted_info = extract_biogrid_interactions(biogrid_data)
    print(json.dumps(extracted_info, indent=2))

[]


In [6]:
url = f"https://webservice.thebiogrid.org/interactions/?accesskey={access_key}"

response = requests.get(url)
response.raise_for_status() # Raise HTTPError for bad responses (4xx or 5xx)
response

<Response [200]>

In [11]:
import requests
import json


gene_name = "MDM2"
url = f"https://webservice.thebiogrid.org/interactions/?accesskey={access_key}&searchNames=true&geneList={gene_name}"

response = requests.get(url)
response.raise_for_status() # Raise HTTPError for bad responses (4xx or 5xx)
tsv_data = response.text

# Split the string into lines, and split lines into columns by tabs
lines = tsv_data.strip().split("\n")
parsed_data = [line.split("\t") for line in lines]


print(parsed_data[:5]) #Show the first 5 entries.

[['244573', '4193', '7157', '110358', '113010', '-', '-', 'MDM2', 'TP53', 'ACTFS|HDMX|hdm2', 'BCC7|LFS1|P53|TRP53', 'Biochemical Activity', 'physical', 'Li M (2003)', '14671306', '9606', '9606', 'Low Throughput', '-', 'Ubiquitination', '-', "Low levels of Mdm2 activity induce monoubiquitination and nuclear export of p53, whereas high levels promote p53's polyubiquitination and nuclear degradation", '-', 'BIOGRID'], ['244620', '9656', '4193', '115014', '110358', 'DAAP-285E11.7', '-', 'MDC1', 'MDM2', 'NFBD1', 'ACTFS|HDMX|hdm2', 'Protein-peptide', 'physical', 'Rodriguez M (2003)', '14578343', '9606', '9606', 'Low Throughput', '-', '-', '-', 'MDC1 BRCT domains bind phosphopeptides with the motif pS-hydrophobic-[EVDI]-[YF];predicted partners', '-', 'BIOGRID'], ['244822', '4193', '1487', '110358', '107869', '-', '-', 'MDM2', 'CTBP1', 'ACTFS|HDMX|hdm2', 'BARS', 'Far Western', 'physical', 'Mirnezami AH (2003)', '12867035', '9606', '9606', 'Low Throughput', '-', '-', '-', 'Repress p53 activity 

In [61]:
import requests
import pandas as pd
import io

gene_name = "CLOCK"
url = f"https://webservice.thebiogrid.org/interactions/?accesskey={access_key}&searchNames=true&geneList={gene_name}"

response = requests.get(url)
response.raise_for_status()  # Raise HTTPError for bad responses (4xx or 5xx)
tsv_data = response.text

# Use io.StringIO to treat the string like a file
tsv_file = io.StringIO(tsv_data)

# Read the data into a DataFrame
df = pd.read_csv(tsv_file, sep="\t", comment="#")

# Optionally, you can rename columns if the header is not provided:
column_names = [
    "BioGRID Interaction ID",
    "Entrez Gene ID for Interactor A",
    "Entrez Gene ID for Interactor B",
    "BioGRID ID for Interactor A",
    "BioGRID ID for Interactor B",
    "Systematic name for Interactor A",
    "Systematic name for Interactor B",
    "Official symbol for Interactor A",
    "Official symbol for Interactor B",
    "Synonyms/Aliases for Interactor A",
    "Synonyms/Aliases for Interactor B",
    "Experimental System Name",
    "Experimental System Type",
    "First author surname of the publication",
    "Pubmed ID of the publication",
    "Organism ID for Interactor A",
    "Organism ID for Interactor B",
    "Interaction Throughput",
    "Quantitative Score",
    "Post Translational Modification",
    "Phenotypes",
    "Qualifications",
    "Tags",
    "Source Database"
]



df.columns = column_names

filtered_column_names =  [
    "Official symbol for Interactor A",
    "Official symbol for Interactor B",
    "Synonyms/Aliases for Interactor A",
    "Synonyms/Aliases for Interactor B",
    "Quantitative Score",
]
df = df[filtered_column_names]

df = df.sort_values("Quantitative Score", ascending=False)

In [62]:


df["Quantitative Score"] = df["Quantitative Score"].replace("-", "0").astype(float)

df = df[df["Quantitative Score"] > 0]

In [64]:
df.shape

(40, 5)